In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer, BatchEncoding, GPT2LMHeadModel, GPT2Config, GPT2ForSequenceClassification
from tqdm import tqdm
import torch as torch
import os
from torch.utils.data import Dataset
import wandb

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
path = "/om2/user/jackking/modular_transformers/scripts/input_statistics/data"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wandb.login(key="a338f755915cccd861b14f29bf68601d8e1ec2c9")

#set seed
seed = 38
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

/om2/user/jackking/anaconda/envs/modular_transformers/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Currently logged in as: jack-g-king (modular_transformers). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jackking/.netrc


In [2]:
class LMDataset(Dataset):
    def __init__(self, inputs, attn_mask=None, labels=None):
        #cast to tensors if not already tensors
        if not torch.is_tensor(inputs):
            inputs = torch.tensor(inputs)
        if not torch.is_tensor(labels):
            labels = torch.tensor(labels)
        if attn_mask is not None and not torch.is_tensor(attn_mask):
            attn_mask = torch.tensor(attn_mask)
            
        self.inputs = inputs
        self.attn_mask = attn_mask
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        if self.labels is None:
            item = {
                'input_ids': self.inputs[idx],
                'attention_mask': self.attn_mask[idx]}
        elif self.attn_mask is None:
            item = {
                'input_ids': self.inputs[idx],
                'labels': self.labels[idx]
            }
        else:
            item = {
                'input_ids': self.inputs[idx],
                'attention_mask': self.attn_mask[idx],
                'labels': self.labels[idx]
            }
        return item

def make_autoregressive_dataset(data):
    tokenizer.pad_token = tokenizer.eos_token
    dataset = tokenizer.batch_encode_plus(data, add_special_tokens=True, padding='longest', return_tensors="pt")
    inputs = dataset["input_ids"]
    attn_mask = dataset["attention_mask"]
    labels = dataset["input_ids"].clone()
    context_len = inputs.size(1)
    return LMDataset(inputs, attn_mask, labels), context_len

def make_classification_dataset(data1, data2):
    tokenizer.pad_token = tokenizer.eos_token
    len1 = len(data1)
    len2 = len(data2)
    combined = data1 + data2
    labels = [0]*len1 + [1]*len2
    dataset = tokenizer.batch_encode_plus(combined, add_special_tokens=True, padding='longest', return_tensors="pt")
    inputs = dataset["input_ids"]
    attn_mask = dataset["attention_mask"]
    context_len = inputs.size(1)
    return LMDataset(inputs, attn_mask, torch.tensor(labels)), context_len

### Dataloading

In [3]:
def get_vocab_size(data):
    blah = []
    for sent in data:
        blah.extend(sent)
    vocab_size = len(set(blah))
    return vocab_size

## Experiment 1

### M1

In [4]:
datatype = "experiment_1"

train_data = pickle.load(open(f"{path}/{datatype}/train_data_A.pkl", 'rb'))
valid_data = pickle.load(open(f"{path}/{datatype}/valid_data_A.pkl", 'rb'))

vocab_size = get_vocab_size(train_data+valid_data)

batch_size = 128

ctx_len = len(train_data[0])

trainset = LMDataset(train_data, labels=train_data)
valset = LMDataset(valid_data, labels=valid_data)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

model_type = "M1"

### M2

##### Bigrams

In [13]:
batch_size = 128
datatype = "experiment_1"
train_data_B1 = pickle.load(open(f"{path}/{datatype}/train_data_B1.pkl", "rb"))
val_data_B1 = pickle.load(open(f"{path}/{datatype}/valid_data_B1.pkl", "rb"))
train_data_B2 = pickle.load(open(f"{path}/{datatype}/train_data_B2.pkl", "rb"))
val_data_B2 = pickle.load(open(f"{path}/{datatype}/valid_data_B2.pkl", "rb"))

train_data = train_data_B1 + train_data_B2
val_data = val_data_B1 + val_data_B2

vocab_size = get_vocab_size(train_data+val_data)
ctx_len = len(train_data[0])

trainset = LMDataset(train_data, labels=train_data)
valset = LMDataset(val_data, labels=val_data)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

model_type = "M2_B"

##### Trigrams

In [17]:
batch_size = 128
datatype = "experiment_1"
train_data_T1 = pickle.load(open(f"{path}/{datatype}/train_data_T1.pkl", "rb"))
val_data_T1 = pickle.load(open(f"{path}/{datatype}/valid_data_T1.pkl", "rb"))
train_data_T2 = pickle.load(open(f"{path}/{datatype}/train_data_T2.pkl", "rb"))
val_data_T2 = pickle.load(open(f"{path}/{datatype}/valid_data_T2.pkl", "rb"))

train_data = train_data_T1 + train_data_T2
val_data = val_data_T1 + val_data_T2

vocab_size = get_vocab_size(train_data+val_data)
ctx_len = len(train_data[0])

trainset = LMDataset(train_data, labels=train_data)
valset = LMDataset(val_data, labels=val_data)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

model_type = "M2_T"

##### Fourgrams

In [25]:
batch_size = 128
datatype = "experiment_1"
train_data_T1 = pickle.load(open(f"{path}/{datatype}/train_data_F1.pkl", "rb"))
val_data_T1 = pickle.load(open(f"{path}/{datatype}/valid_data_F1.pkl", "rb"))
train_data_T2 = pickle.load(open(f"{path}/{datatype}/train_data_F2.pkl", "rb"))
val_data_T2 = pickle.load(open(f"{path}/{datatype}/valid_data_F2.pkl", "rb"))

train_data = train_data_T1 #+ train_data_T2
val_data = val_data_T1 #+ val_data_T2

vocab_size = get_vocab_size(train_data+val_data)
ctx_len = len(train_data[0])

trainset = LMDataset(train_data, labels=train_data)
valset = LMDataset(val_data, labels=val_data)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

model_type = "M2_F1"

### M3

##### Bigrams

In [ ]:
batch_size = 8
datatype = "longer_data"
train_data_B1 = pickle.load(open(f"{path}/{datatype}/train_data_B1.pkl", "rb"))
val_data_B1 = pickle.load(open(f"{path}/{datatype}/valid_data_B1.pkl", "rb"))
train_data_B2 = pickle.load(open(f"{path}/{datatype}/train_data_B2.pkl", "rb"))
val_data_B2 = pickle.load(open(f"{path}/{datatype}/valid_data_B2.pkl", "rb"))

len1 = len(train_data_B1)
len2 = len(train_data_B2)
train_data = train_data_B1 + train_data_B2
train_labels = [0]*len1 + [1]*len2
val_data = val_data_B1 + val_data_B2
len1 = len(val_data_B1)
len2 = len(val_data_B2)
val_labels = [0]*len1 + [1]*len2

ctx_len = len(train_data[0])

vocab_size = get_vocab_size(train_data+val_data)

trainset = LMDataset(train_data, labels=torch.tensor(train_labels))
valset = LMDataset(val_data, labels=torch.tensor(val_labels))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False) 

model_type = "M3"

##### Trigrams

In [8]:
batch_size = 128
datatype = "longer_data"
train_data_T1 = pickle.load(open(f"{path}/{datatype}/train_data_T1.pkl", "rb"))
val_data_T1 = pickle.load(open(f"{path}/{datatype}/valid_data_T1.pkl", "rb"))
train_data_T2 = pickle.load(open(f"{path}/{datatype}/train_data_T2.pkl", "rb"))
val_data_T2 = pickle.load(open(f"{path}/{datatype}/valid_data_T2.pkl", "rb"))

len1 = len(train_data_T1)
len2 = len(train_data_T2)
train_data = train_data_T1 + train_data_T2
train_labels = [0]*len1 + [1]*len2
val_data = val_data_T1 + val_data_T2
len1 = len(val_data_T1)
len2 = len(val_data_T2)
val_labels = [0]*len1 + [1]*len2

ctx_len = len(train_data[0])

vocab_size = get_vocab_size(train_data+val_data)

trainset = LMDataset(train_data, labels=torch.tensor(train_labels))
valset = LMDataset(val_data, labels=torch.tensor(val_labels))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False) 

model_type = "M3_T"

##### Fourgrams

In [4]:
batch_size = 128
datatype = "longer_data"
train_data_T1 = pickle.load(open(f"{path}/{datatype}/train_data_F1.pkl", "rb"))
val_data_T1 = pickle.load(open(f"{path}/{datatype}/valid_data_F1.pkl", "rb"))
train_data_T2 = pickle.load(open(f"{path}/{datatype}/train_data_F2.pkl", "rb"))
val_data_T2 = pickle.load(open(f"{path}/{datatype}/valid_data_F2.pkl", "rb"))

len1 = len(train_data_T1)
len2 = len(train_data_T2)
train_data = train_data_T1 + train_data_T2
train_labels = [0]*len1 + [1]*len2
val_data = val_data_T1 + val_data_T2
len1 = len(val_data_T1)
len2 = len(val_data_T2)
val_labels = [0]*len1 + [1]*len2

ctx_len = len(train_data[0])

vocab_size = get_vocab_size(train_data+val_data)

trainset = LMDataset(train_data, labels=torch.tensor(train_labels))
valset = LMDataset(val_data, labels=torch.tensor(val_labels))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False) 

model_type = "M3_F"

## experiment 2

#### M1

In [12]:
datacategory = "C"

In [13]:
datatype = f"experiment_2S_{datacategory}"
data_path = f"{path}/experiment_2S"

batch_size = 128

train_data = pickle.load(open(f"{data_path}/train_data_{datacategory}.pkl", 'rb'))
valid_data = pickle.load(open(f"{data_path}/valid_data_{datacategory}.pkl", 'rb'))

vocab_size = get_vocab_size(train_data["inputs"]+valid_data["inputs"])

ctx_len = len(train_data["inputs"][0])

trainset = LMDataset(train_data["inputs"], labels=train_data["inputs"])
valset = LMDataset(valid_data["inputs"], labels=valid_data["inputs"])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

model_type = "M1"

#### M2

In [ ]:
datatype = f"experiment_2_{datacategory}"
data_path = f"{path}/experiment_2"

train_data = pickle.load(open(f"{data_path}/train_data_{datacategory}.pkl", 'rb'))
valid_data = pickle.load(open(f"{data_path}/valid_data_{datacategory}.pkl", 'rb'))

ctx_len = len(train_data["inputs"][0])

vocab_size = get_vocab_size(train_data["inputs"] + valid_data["inputs"])

batch_size = 128

num_labels = 200

trainset = LMDataset(train_data["inputs"], labels=train_data["labels"])
valset = LMDataset(valid_data["inputs"], labels=valid_data["labels"])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

model_type = "M2"

### training loop

In [6]:
def evaluate(model, valloader):
    model.eval()
    losses = []
    for step, batch in tqdm(enumerate(valloader), total=len(valloader)):
        with torch.no_grad():
            inputs = batch["input_ids"].to(device)
            if "attention_mask" in batch:
                attention_mask = batch["attention_mask"].to(device)
            else:
                attention_mask = None
            labels = batch["labels"].to(device)
            outputs = model(inputs, labels=labels, attention_mask=attention_mask)
        losses.append(outputs.loss)
    loss = torch.mean(torch.stack(losses))
    return loss.item()

In [14]:
def train(model, optimizer, lr_scheduler, train_config, model_name, trainloader, valloader):
    wandb.init(project="input statistics", config=train_config)
    run_name = wandb.run.name

    save_epochs = train_config["num_epochs"] // 10
    save_epochs = 1

    for epoch in tqdm(range(train_config["num_epochs"])):
        model.train()
        torch.cuda.empty_cache()
        for step, batch in tqdm(enumerate(trainloader), total=len(trainloader)):
            optimizer.zero_grad()
            inputs = batch["input_ids"].to(device)
            if "attention_mask" in batch:
                attention_mask = batch["attention_mask"].to(device)
            else:
                attention_mask = None
            labels = batch["labels"].to(device)

            # print(inputs[0], labels[0])

            outputs = model(inputs, labels=labels, attention_mask=attention_mask)
            loss = outputs.loss 
            loss.backward()
            if train_config["lr_scheduler"] is not None:
                lr_scheduler.step()
            optimizer.step()

            wandb.log({"step": step + len(trainloader) * epoch})
            wandb.log({"loss": loss.item()})
            wandb.log({"learning_rate": optimizer.param_groups[0]['lr']})

        wandb.log({"epoch": epoch})
        val_loss = evaluate(model, valloader)
        wandb.log({"val_loss": val_loss})

        #save model
        if epoch % save_epochs == 0:
            model_dir = os.path.join(path, model_name, run_name, f"epoch_{epoch}")
            os.makedirs(model_dir, exist_ok=True)
            model.save_pretrained(model_dir)

    wandb.finish()

    #save model
    model_dir = os.path.join(path, model_name, run_name, "final_chkpoint")
    os.makedirs(model_dir, exist_ok=True)
    model.save_pretrained(model_dir)


In [15]:
embedding_dim = 128
n_layer = 12
n_head = 4
resid_pdrop = 0.1
embd_pdrop = 0.2
attn_pdrop = 0.2
tokenizer.pad_token = tokenizer.eos_token
vocab_size = vocab_size + 5 #for special tokens
num_labels = 200

model_config = GPT2Config(n_layer = n_layer, n_head = n_head, n_embd = embedding_dim, n_positions = ctx_len, #vocab_size = vocab_size,
                          resid_pdrop=resid_pdrop, embd_pdrop=embd_pdrop, attn_pdrop=attn_pdrop, num_labels=num_labels
                          )
# model = GPT2ForSequenceClassification._from_config(model_config)
model = GPT2LMHeadModel._from_config(model_config)

model.config.pad_token_id = model.config.eos_token_id
model.to(device)

lr_scheduler = None

model_name = f"{datatype}/{model_type}_{embedding_dim}_{n_layer}"
train_config = {"num_epochs": 50, "lr": 0.0001, "lr_scheduler": lr_scheduler, "batch_size": batch_size, "resid_pdrop": resid_pdrop, "embd_pdrop": embd_pdrop, "n_head": n_head,
                "attn_pdrop": attn_pdrop, "model_name": model_name, "model_type": model_type, "embedding_dim": embedding_dim, "n_layer": n_layer, "ctx_len": ctx_len, "datatype": datatype}

optimizer = torch.optim.Adam(model.parameters(), lr=train_config["lr"])
if train_config["lr_scheduler"] is not None:
    if train_config["lr_scheduler"] == "cosine_annealing":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"]*30)
    elif train_config["lr_scheduler"] == "cosine":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"] * len(trainloader))
    elif train_config["lr_scheduler"] == "step":
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.98)

train(model, optimizer, lr_scheduler, train_config, model_name, trainloader, valloader)

100%|██████████| 50/50 [14:53<00:00, 17.86s/it]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▆▅▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▆▆▅▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,49
learning_rate,0.0001
loss,0.64896
step,5899
val_loss,0.64514


In [7]:
batch_size = 128
datatype = "experiment_1"
datacategory = "EB"

train_data_1 = pickle.load(open(f"{path}/{datatype}/train_data_{datacategory}1.pkl", "rb"))
val_data_1 = pickle.load(open(f"{path}/{datatype}/valid_data_{datacategory}1.pkl", "rb"))
train_data_2 = pickle.load(open(f"{path}/{datatype}/train_data_{datacategory}2.pkl", "rb"))
val_data_2 = pickle.load(open(f"{path}/{datatype}/valid_data_{datacategory}2.pkl", "rb"))

train_data = train_data_1 + train_data_2
val_data = val_data_1 + val_data_2

len1 = len(train_data_1)
len2 = len(train_data_2)
train_labels = [0]*len1 + [1]*len2
len1 = len(val_data_1)
len2 = len(val_data_2)
val_labels = [0]*len1 + [1]*len2

ctx_len = len(train_data[0])

vocab_size = get_vocab_size(train_data+val_data)

trainset = LMDataset(train_data, labels=torch.tensor(train_labels))
valset = LMDataset(val_data, labels=torch.tensor(val_labels))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False) 

model_type = f"M3_{datacategory}"

embedding_dim = 128
n_layer = 12
n_head = 4
resid_pdrop = 0.1
embd_pdrop = 0.3
attn_pdrop = 0.3
tokenizer.pad_token = tokenizer.eos_token
num_labels = 2

model_config = GPT2Config(n_layer = n_layer, n_head = n_head, n_embd = embedding_dim, n_positions = ctx_len, #vocab_size = vocab_size,
                          resid_pdrop=resid_pdrop, embd_pdrop=embd_pdrop, attn_pdrop=attn_pdrop, num_labels=num_labels
                          )
model = GPT2ForSequenceClassification._from_config(model_config)

model.config.pad_token_id = model.config.eos_token_id
model.to(device)

lr_scheduler = None

model_name = f"{datatype}/{model_type}_{embedding_dim}_{n_layer}"
train_config = {"num_epochs": 200, "lr": 0.000005, "lr_scheduler": lr_scheduler, "batch_size": batch_size, "resid_pdrop": resid_pdrop, "embd_pdrop": embd_pdrop, "n_head": n_head,
                "attn_pdrop": attn_pdrop, "model_name": model_name, "model_type": model_type, "embedding_dim": embedding_dim, "n_layer": n_layer, "ctx_len": ctx_len, "datatype": datatype}

optimizer = torch.optim.Adam(model.parameters(), lr=train_config["lr"])
if train_config["lr_scheduler"] is not None:
    if train_config["lr_scheduler"] == "cosine_annealing":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"]*30)
    elif train_config["lr_scheduler"] == "cosine":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"] * len(trainloader))
    elif train_config["lr_scheduler"] == "step":
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.98)

train(model, optimizer, lr_scheduler, train_config, model_name, trainloader, valloader)

  0%|          | 0/200 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.
100%|██████████| 200/200 [34:13<00:00, 10.27s/it]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
learning_rate,1e-05
loss,0.0
step,31399
val_loss,0.0


In [31]:
batch_size = 128
datatype = "experiment_1"
datacategory = "EB"

train_data_1 = pickle.load(open(f"{path}/{datatype}/train_data_{datacategory}1.pkl", "rb"))
val_data_1 = pickle.load(open(f"{path}/{datatype}/valid_data_{datacategory}1.pkl", "rb"))
train_data_2 = pickle.load(open(f"{path}/{datatype}/train_data_{datacategory}2.pkl", "rb"))
val_data_2 = pickle.load(open(f"{path}/{datatype}/valid_data_{datacategory}2.pkl", "rb"))

train_data = train_data_1 + train_data_2
val_data = val_data_1 + val_data_2

len1 = len(train_data_1)
len2 = len(train_data_2)
train_labels = [0]*len1 + [1]*len2
len1 = len(val_data_1)
len2 = len(val_data_2)
val_labels = [0]*len1 + [1]*len2

ctx_len = len(train_data[0])

vocab_size = get_vocab_size(train_data+val_data)

trainset = LMDataset(train_data, labels=torch.tensor(train_labels))
valset = LMDataset(val_data, labels=torch.tensor(val_labels))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False) 

model_type = f"M3_{datacategory}"

embedding_dim = 128
n_layer = 12
n_head = 4
resid_pdrop = 0.1
embd_pdrop = 0.3
attn_pdrop = 0.3
tokenizer.pad_token = tokenizer.eos_token
num_labels = 2

model_config = GPT2Config(n_layer = n_layer, n_head = n_head, n_embd = embedding_dim, n_positions = ctx_len, #vocab_size = vocab_size,
                          resid_pdrop=resid_pdrop, embd_pdrop=embd_pdrop, attn_pdrop=attn_pdrop, num_labels=num_labels
                          )
model = GPT2ForSequenceClassification._from_config(model_config)

model.config.pad_token_id = model.config.eos_token_id
model.to(device)

lr_scheduler = None

model_name = f"{datatype}/{model_type}_{embedding_dim}_{n_layer}"
train_config = {"num_epochs": 300, "lr": 0.000005, "lr_scheduler": lr_scheduler, "batch_size": batch_size, "resid_pdrop": resid_pdrop, "embd_pdrop": embd_pdrop, "n_head": n_head,
                "attn_pdrop": attn_pdrop, "model_name": model_name, "model_type": model_type, "embedding_dim": embedding_dim, "n_layer": n_layer, "ctx_len": ctx_len, "datatype": datatype}

optimizer = torch.optim.Adam(model.parameters(), lr=train_config["lr"])
if train_config["lr_scheduler"] is not None:
    if train_config["lr_scheduler"] == "cosine_annealing":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"]*30)
    elif train_config["lr_scheduler"] == "cosine":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"] * len(trainloader))
    elif train_config["lr_scheduler"] == "step":
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.98)

train(model, optimizer, lr_scheduler, train_config, model_name, trainloader, valloader)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▄▂▂▃▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_loss,▄▁▃▁▂▂█▁▁▂▁▁▃▂▂▂▃▁▃
epoch,18
learning_rate,5e-05
loss,0.00378
step,3037
val_loss,0.19453


100%|██████████| 300/300 [51:28<00:00, 10.30s/it]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
val_loss,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,299
learning_rate,1e-05
loss,0.0
step,47099
val_loss,0.0


In [11]:
batch_size = 128
datatype = "experiment_1"
datacategory = "EB"

train_data_1 = pickle.load(open(f"{path}/{datatype}/train_data_{datacategory}1.pkl", "rb"))
val_data_1 = pickle.load(open(f"{path}/{datatype}/valid_data_{datacategory}1.pkl", "rb"))
train_data_2 = pickle.load(open(f"{path}/{datatype}/train_data_{datacategory}2.pkl", "rb"))
val_data_2 = pickle.load(open(f"{path}/{datatype}/valid_data_{datacategory}2.pkl", "rb"))

train_data = train_data_1 + train_data_2
val_data = val_data_1 + val_data_2

vocab_size = get_vocab_size(train_data+val_data)
ctx_len = len(train_data[0])

trainset = LMDataset(train_data, labels=train_data)
valset = LMDataset(val_data, labels=val_data)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

model_type = f"M2_{datacategory}"

embedding_dim = 128
n_layer = 12
n_head = 4
resid_pdrop = 0.1
embd_pdrop = 0.2
attn_pdrop = 0.2
tokenizer.pad_token = tokenizer.eos_token
num_labels = 200

model_config = GPT2Config(n_layer = n_layer, n_head = n_head, n_embd = embedding_dim, n_positions = ctx_len, #vocab_size = vocab_size,
                          resid_pdrop=resid_pdrop, embd_pdrop=embd_pdrop, attn_pdrop=attn_pdrop, num_labels=num_labels
                          )
# model = GPT2ForSequenceClassification._from_config(model_config)
model = GPT2LMHeadModel._from_config(model_config)

model.config.pad_token_id = model.config.eos_token_id
model.to(device)

lr_scheduler = None

model_name = f"{datatype}/{model_type}_{embedding_dim}_{n_layer}"
train_config = {"num_epochs": 150, "lr": 0.0001, "lr_scheduler": lr_scheduler, "batch_size": batch_size, "resid_pdrop": resid_pdrop, "embd_pdrop": embd_pdrop, "n_head": n_head,
                "attn_pdrop": attn_pdrop, "model_name": model_name, "model_type": model_type, "embedding_dim": embedding_dim, "n_layer": n_layer, "ctx_len": ctx_len, "datatype": datatype}

optimizer = torch.optim.Adam(model.parameters(), lr=train_config["lr"])
if train_config["lr_scheduler"] is not None:
    if train_config["lr_scheduler"] == "cosine_annealing":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"]*30)
    elif train_config["lr_scheduler"] == "cosine":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"] * len(trainloader))
    elif train_config["lr_scheduler"] == "step":
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.98)

train(model, optimizer, lr_scheduler, train_config, model_name, trainloader, valloader)

100%|██████████| 150/150 [44:02<00:00, 17.61s/it]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▇▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,149
learning_rate,0.0001
loss,0.62149
step,17699
val_loss,0.6465


In [23]:
batch_size = 128
datatype = "experiment_1"
train_data_T1 = pickle.load(open(f"{path}/{datatype}/train_data_F1.pkl", "rb"))
val_data_T1 = pickle.load(open(f"{path}/{datatype}/valid_data_F1.pkl", "rb"))
train_data_T2 = pickle.load(open(f"{path}/{datatype}/train_data_F2.pkl", "rb"))
val_data_T2 = pickle.load(open(f"{path}/{datatype}/valid_data_F2.pkl", "rb"))

train_data = train_data_T1 + train_data_T2
val_data = val_data_T1 + val_data_T2

vocab_size = get_vocab_size(train_data+val_data)
ctx_len = len(train_data[0])

trainset = LMDataset(train_data, labels=train_data)
valset = LMDataset(val_data, labels=val_data)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

model_type = "M2_F"

In [24]:
embedding_dim = 128
n_layer = 12
n_head = 4
resid_pdrop = 0.1
embd_pdrop = 0.2
attn_pdrop = 0.2
tokenizer.pad_token = tokenizer.eos_token
num_labels = 200

model_config = GPT2Config(n_layer = n_layer, n_head = n_head, n_embd = embedding_dim, n_positions = ctx_len, #vocab_size = vocab_size,
                          resid_pdrop=resid_pdrop, embd_pdrop=embd_pdrop, attn_pdrop=attn_pdrop, num_labels=num_labels
                          )
# model = GPT2ForSequenceClassification._from_config(model_config)
model = GPT2LMHeadModel._from_config(model_config)

model.config.pad_token_id = model.config.eos_token_id
model.to(device)

lr_scheduler = None

model_name = f"{datatype}/{model_type}_{embedding_dim}_{n_layer}"
train_config = {"num_epochs": 100, "lr": 0.001, "lr_scheduler": lr_scheduler, "batch_size": batch_size, "resid_pdrop": resid_pdrop, "embd_pdrop": embd_pdrop, "n_head": n_head,
                "attn_pdrop": attn_pdrop, "model_name": model_name, "model_type": model_type, "embedding_dim": embedding_dim, "n_layer": n_layer, "ctx_len": ctx_len, "datatype": datatype}

optimizer = torch.optim.Adam(model.parameters(), lr=train_config["lr"])
if train_config["lr_scheduler"] is not None:
    if train_config["lr_scheduler"] == "cosine_annealing":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"]*30)
    elif train_config["lr_scheduler"] == "cosine":
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_config["num_epochs"] * len(trainloader))
    elif train_config["lr_scheduler"] == "step":
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.98)

train(model, optimizer, lr_scheduler, train_config, model_name, trainloader, valloader)

100%|██████████| 100/100 [38:18<00:00, 22.99s/it]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
learning_rate,0.001
loss,3.18818
step,15699
val_loss,4.42785
